# 📊 Módulo 01a - Extracción y Transformación INE

## 🎯 Objetivo
Extraer y transformar las **13 tablas del INE** necesarias para el análisis de desigualdad social.

## 📋 Tablas procesadas
1. IPC General Nacional (24077)
2. Umbral de Pobreza por Hogar (11205_4)
3. Carencia Material por Decil (9973)
4. AROPE y Componentes (Edad/Sexo: 29287, Hogar: 60259, Laboral: 74862)
5. Desigualdad: Gini y S80/S20 por CCAA (60143)
6. Renta Media por Decil (11106_2)
7. Población por Edad, Sexo y Nacionalidad (56936)
8. Población por CCAA, Edad y Sexo (66014)
9. AROPE por CCAA (29288)
10. Gasto Medio por Hogar por Quintil - EPF (24900)
11. IPC Sectorial por Grupos ECOICOP (50902)

## 📤 Output
Genera **14 archivos pickle** en `outputs/pickle_cache/` con todos los DataFrames INE.

---

### 1. Imports y Configuración

In [1]:
import requests
import pandas as pd
import re
import pickle
from pathlib import Path
from datetime import datetime

# Crear directorio para cache si no existe
CACHE_DIR = Path('../../outputs/pickle_cache')
CACHE_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Imports cargados")
print(f"📁 Cache directory: {CACHE_DIR.absolute()}")
print(f"🕒 Inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Imports cargados
📁 Cache directory: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\outputs\pickle_cache
🕒 Inicio: 2025-11-16 14:14:34


### 📈 2. IPC General Nacional (INE - Tabla 24077)

**¿Qué es?**  
El **Índice de Precios de Consumo (IPC)** mide cómo cambian los precios de los bienes y servicios.

**Transformación:** Datos mensuales → Promedio anual → Cálculo de inflación (%)

In [2]:

codigo_ipc = "24077"
url_ipc = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{codigo_ipc}"
response_ipc = requests.get(url_ipc)
data_ipc = response_ipc.json()

datos_ipc_limpios = []
df_ipc_raw = pd.DataFrame(data_ipc)

for idx, row in df_ipc_raw.iterrows():
    if isinstance(row['Data'], list):
        for dato in row['Data']:
            datos_ipc_limpios.append({
                'Periodo': dato.get('Anyo', dato.get('NombrePeriodo', None)),
                'IPC_Indice': dato.get('Valor', None)
            })

df_ipc_limpio = pd.DataFrame(datos_ipc_limpios)
df_ipc_limpio['IPC_Indice'] = pd.to_numeric(df_ipc_limpio['IPC_Indice'], errors='coerce')
df_ipc_limpio['Periodo'] = df_ipc_limpio['Periodo'].astype(str)

df_ipc_limpio['Año'] = pd.to_numeric(df_ipc_limpio['Periodo'], errors='coerce').astype('Int64')
df_ipc_anual = df_ipc_limpio.groupby('Año', as_index=False).agg({
    'IPC_Indice': 'mean'
}).rename(columns={'IPC_Indice': 'IPC_Medio_Anual'})
df_ipc_anual['IPC_Medio_Anual'] = df_ipc_anual['IPC_Medio_Anual'].round(3)

df_ipc_anual['Inflacion_Anual_%'] = df_ipc_anual['IPC_Medio_Anual'].pct_change() * 100
df_ipc_anual['Inflacion_Anual_%'] = df_ipc_anual['Inflacion_Anual_%'].round(2)

print(f"  > Completado. Registros anuales: {len(df_ipc_anual)}")


  > Completado. Registros anuales: 24


### 🏠 3. Umbral de Pobreza por Hogar (INE - Tabla 11205_4)

**¿Qué es?**  
Umbral de pobreza según tipo de hogar (60% mediana nacional).

**Transformación:** Datos por hogar → Formato largo

In [3]:

ruta_tabla_umbral = "t00/ICV/dim1/l0/11205_4.px"
url_umbral = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{ruta_tabla_umbral}"
response_umbral = requests.get(url_umbral)
data_umbral = response_umbral.json()

datos_limpios_umbral = []
df_umbral_raw = pd.DataFrame(data_umbral)

for idx, row in df_umbral_raw.iterrows():
    tipo_hogar = row['Nombre']
    for dato in row['Data']:
        datos_limpios_umbral.append({
            'Tipo_Hogar': tipo_hogar,
            'Año': int(dato['NombrePeriodo']),
            'Umbral_Euros': float(dato['Valor'])
        })

df_umbral_limpio = pd.DataFrame(datos_limpios_umbral)
df_umbral_limpio = df_umbral_limpio.sort_values(['Año', 'Tipo_Hogar'], ascending=[False, True])

print(f"  > Completado. Registros: {len(df_umbral_limpio)}")


  > Completado. Registros: 32


### 🍽️ 4. Carencia Material por Decil (INE - Tabla 9973)

**¿Qué es?**  
Porcentaje de personas que NO pueden permitirse bienes/servicios básicos, por decil de renta.

In [4]:

tabla_carencia = "9973"
url_carencia = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{tabla_carencia}"
response_carencia = requests.get(url_carencia)
data_carencia = response_carencia.json()

datos_limpios_carencia = []
df_carencia_raw = pd.DataFrame(data_carencia)

for idx, row in df_carencia_raw.iterrows():
    nombre_completo = row.get('Nombre', '')
    
    item_match = re.match(r'^(.+?)\\.\\s*Total Nacional', nombre_completo)
    nombre_item = item_match.group(1).strip() if item_match else nombre_completo
    
    deciles_map = {
        'Primer decil': 'D1', 'Segundo decil': 'D2', 'Tercer decil': 'D3',
        'Cuarto decil': 'D4', 'Quinto decil': 'D5', 'Sexto decil': 'D6',
        'Séptimo decil': 'D7', 'Septimo decil': 'D7', 'Octavo decil': 'D8',
        'Noveno decil': 'D9', 'Décimo decil': 'D10', 'Decimo decil': 'D10'
    }
    decil = 'Total Nacional'
    for k, v in deciles_map.items():
        if k in nombre_completo:
            decil = v
            break
            
    if isinstance(row['Data'], list):
        for dato in row['Data']:
            datos_limpios_carencia.append({
                'Item': nombre_item,
                'Año': dato.get('Anyo', None),
                'Valor': dato.get('Valor', None),
                'Decil': decil
            })

df_carencia_material = pd.DataFrame(datos_limpios_carencia)
df_carencia_material['Valor'] = df_carencia_material['Valor'].astype(str).str.replace(',', '.').astype(float)
df_carencia_material['Año'] = pd.to_numeric(df_carencia_material['Año'], errors='coerce').astype('Int64')
df_carencia_material = df_carencia_material.dropna(subset=['Valor', 'Año'])

print(f"  > Completado. Registros: {len(df_carencia_material)}")


  > Completado. Registros: 1683


### 🌍 5. Tasa AROPE y Componentes (INE - 3 tablas)

**AROPE** = At Risk Of Poverty or social Exclusion. Mide exclusión social usando 3 criterios.

**Tablas:** Edad/Sexo (29287), Hogar (60259), Laboral (74862)

In [5]:


# --- 2.4.1 AROPE por Edad y Sexo (29287) ---
tabla_arope_edad_sexo = "29287"
url_arope_edad_sexo = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{tabla_arope_edad_sexo}"
response_arope = requests.get(url_arope_edad_sexo)
data_arope_raw = response_arope.json()

arope_records = []
for record in data_arope_raw:
    nombre = record.get('Nombre', '')
    if 'Data' in record and isinstance(record['Data'], list):
        for data_point in record['Data']:
            year = data_point.get('Anyo')
            valor = data_point.get('Valor')
            if year is not None and valor is not None:
                sexo = 'Hombre' if 'Hombre' in nombre else 'Mujer' if 'Mujer' in nombre else 'Total'
                
                if 'Total' in nombre and not any(x in nombre for x in ['Menos de 16', '16 a 29', '30 a 44', '45 a 64', '65 y más', 'Menos de 18', '18 a 64']):
                    edad = 'Total'
                elif 'Menos de 16' in nombre or 'Menores de 16' in nombre:
                    edad = 'Menores de 16 años'
                elif '16 a 29' in nombre:
                    edad = '16 a 29 años'
                elif '30 a 44' in nombre:
                    edad = '30 a 44 años'
                elif '45 a 64' in nombre:
                    edad = '45 a 64 años'
                elif '65 y más' in nombre or '65 y +' in nombre:
                    edad = '65 y más años'
                elif 'Menos de 18' in nombre or 'Menores de 18' in nombre:
                    edad = 'Menos de 18 años'
                elif '18 a 64' in nombre:
                    edad = '18 a 64 años'
                else:
                    edad = 'Total'
                
                indicador = 'AROP' if 'En riesgo de pobreza' in nombre and 'AROPE' not in nombre else 'Carencia Material Severa' if 'Carencia material' in nombre else 'Baja Intensidad Laboral' if 'Baja intensidad' in nombre else 'AROPE'
                
                arope_records.append({
                    'Año': int(year), 'Sexo': sexo, 'Edad': edad,
                    'Indicador': indicador, 'Valor': float(valor)
                })
df_arope_edad_sexo = pd.DataFrame(arope_records)
df_arope_edad_sexo = df_arope_edad_sexo.drop_duplicates(subset=['Año', 'Sexo', 'Edad'], keep='first')
print(f"  > AROPE Edad/Sexo: {len(df_arope_edad_sexo)} registros")

# --- 2.4.2 AROPE por Tipo de Hogar (60259) ---
tabla_hogar = "60259"
url_tabla_hogar = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{tabla_hogar}"
response_hogar = requests.get(url_tabla_hogar)
data_hogar_raw = response_hogar.json()

hogar_records = []
for record in data_hogar_raw:
    nombre = record.get('Nombre', '')
    
    if 'Total' in nombre and not any(x in nombre for x in ['1 adulto', '2 adultos', 'Otros', 'No consta', 'Hogares de una persona']):
        tipo_hogar = 'Total'
    elif '1 adulto con 1 ó más niños dependientes' in nombre or '1 adulto con 1 o más niños dependientes' in nombre:
        tipo_hogar = '1 adulto con 1 o más niños dependientes'
    elif '2 adultos con 1 ó más niños dependientes' in nombre or '2 adultos con 1 o más niños dependientes' in nombre:
        tipo_hogar = '2 adultos con 1 o más niños dependientes'
    elif '2 adultos sin niños dependientes' in nombre:
        tipo_hogar = '2 adultos sin niños dependientes'
    elif 'Hogares de una persona' in nombre:
        tipo_hogar = 'Hogares de una persona'
    elif 'Otros hogares con niños dependientes' in nombre:
        tipo_hogar = 'Otros hogares con niños dependientes'
    elif 'Otros hogares sin niños dependientes' in nombre:
        tipo_hogar = 'Otros hogares sin niños dependientes'
    elif 'No consta' in nombre:
        tipo_hogar = 'No consta'
    else:
        tipo_hogar = 'Desconocido'
    
    indicador = 'AROP' if 'En riesgo de pobreza' in nombre and 'exclusión' not in nombre.lower() else 'Carencia Material Severa' if 'carencia material' in nombre.lower() else 'Baja Intensidad Laboral' if 'baja intensidad' in nombre.lower() else 'AROPE'
    
    if 'Data' in record and isinstance(record['Data'], list):
        for data_point in record['Data']:
            year = data_point.get('Anyo')
            valor = data_point.get('Valor')
            if year is not None and valor is not None:
                hogar_records.append({
                    'Año': int(year), 'Tipo_Hogar': tipo_hogar,
                    'Indicador': indicador, 'Valor': float(valor)
                })
df_arope_hogar = pd.DataFrame(hogar_records)
df_arope_hogar = df_arope_hogar[df_arope_hogar['Tipo_Hogar'] != 'No consta'].copy()
print(f"  > AROPE Hogar: {len(df_arope_hogar)} registros (sin 'No consta')")

# --- 2.4.3 AROPE por Situación Laboral (74862) ---
tabla_laboral = "74862"
url_tabla_laboral = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{tabla_laboral}"
response_laboral = requests.get(url_tabla_laboral)
data_laboral_raw = response_laboral.json()

registros_laboral = []
for record in data_laboral_raw:
    nombre = record.get('Nombre', '')
    nombre_parts = [p.strip() for p in nombre.split('.')]
    
    sexo = nombre_parts[0] if len(nombre_parts) > 0 else 'Total'
    situacion_laboral = nombre_parts[1] if len(nombre_parts) > 1 else 'Total'
    territorio = 'UE-27' if 'UE27' in nombre_parts[2] else 'España' if 'Total Nacional' in nombre_parts[2] else nombre_parts[2]
    
    datos_valores = record.get('Data', [])
    if isinstance(datos_valores, list):
        for valor_obj in datos_valores:
            if isinstance(valor_obj, dict):
                anyo = valor_obj.get('Anyo')
                valor = valor_obj.get('Valor')
                if anyo is not None and valor is not None:
                    registros_laboral.append({
                        'Sexo': sexo, 'Situacion_Laboral': situacion_laboral,
                        'Territorio': territorio, 'Año': int(anyo), 'AROPE': float(valor)
                    })
df_arope_laboral = pd.DataFrame(registros_laboral)
print(f"  > AROPE Laboral: {len(df_arope_laboral)} registros")


  > AROPE Edad/Sexo: 408 registros
  > AROPE Hogar: 308 registros (sin 'No consta')


  > AROPE Laboral: 152 registros


### ⚖️ 7. Desigualdad: Gini y S80/S20 por CCAA (INE - Tabla 60143)

**¿Qué es?**  
Dos indicadores clave para medir desigualdad de ingresos:

#### 📊 Coeficiente de Gini
- **Rango**: 0 (igualdad perfecta) a 1 (máxima desigualdad)
- **Contexto España**: Típicamente entre 0.30 - 0.35

#### 💰 Ratio S80/S20
- **Cálculo**: Renta del 20% más RICO ÷ Renta del 20% más POBRE
- **Ejemplo**: S80/S20 = 6.5 → Los ricos ganan 6.5 veces más que los pobres

**Transformación:** Datos por CCAA → Filtrar Gini y S80/S20 (excluir imputados) → Pivotado

In [6]:

tabla_gini = "60143"
url_tabla_gini = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{tabla_gini}"
response_gini = requests.get(url_tabla_gini)
data_gini_raw = response_gini.json()

registros_desigualdad = []
for record in data_gini_raw:
    nombre = record.get('Nombre', '')
    nombre_lower = nombre.lower()
    
    indicador = 'Gini' if 'gini' in nombre_lower and 'imputado' not in nombre_lower else 'S80/S20' if 's80' in nombre_lower and 'imputado' not in nombre_lower else None
    if indicador is None: continue
        
    territorio = nombre.split('.')[0].strip()
    
    datos_valores = record.get('Data', [])
    if isinstance(datos_valores, list):
        for valor_obj in datos_valores:
            if isinstance(valor_obj, dict):
                anyo = valor_obj.get('Anyo')
                valor = valor_obj.get('Valor')
                if anyo is not None and valor is not None:
                    registros_desigualdad.append({
                        'Territorio': territorio, 'Indicador': indicador,
                        'Año': int(anyo), 'Valor': float(valor)
                    })

df_desigualdad = pd.DataFrame(registros_desigualdad)
df_gini_ccaa = df_desigualdad.pivot_table(
    index=['Territorio', 'Año'], columns='Indicador', values='Valor'
).reset_index()

print(f"  > Completado. Registros: {len(df_gini_ccaa)}")


  > Completado. Registros: 340


### 💰 8. Renta Media por Decil (INE - Tabla 11106_2)

**¿Qué es?**  
Renta media y mediana de la población dividida en 10 grupos (deciles) del más pobre al más rico.

| Decil | Representa |
|-------|-----------|
| **D1** | 10% más pobre |
| **D2-D9** | Clases medias |
| **D10** | 10% más rico |

**Transformación:** Datos por decil → Extracción de indicador y decil → Pivotado

In [7]:

ruta_tabla_renta = "t00/ICV/dim1/l0/11106_2.px"
url_renta_decil = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{ruta_tabla_renta}"
response_renta_decil = requests.get(url_renta_decil)
data_renta_decil_raw = response_renta_decil.json()

registros_deciles = []
mapeo_deciles = {
    'Total': 'Total', 'Primer decil': 'D1', 'Segundo decil': 'D2', 'Tercer decil': 'D3',
    'Cuarto decil': 'D4', 'Quinto decil': 'D5', 'Sexto decil': 'D6', 'Séptimo decil': 'D7',
    'Octavo decil': 'D8', 'Noveno decil': 'D9', 'Décimo decil': 'D10'
}

for record in data_renta_decil_raw:
    nombre = record.get('Nombre', '')
    partes = [p.strip() for p in nombre.split(',')]
    if len(partes) >= 2:
        indicador = partes[0]
        decil_nombre = partes[1]
        decil = mapeo_deciles.get(decil_nombre, decil_nombre)
        
        datos_valores = record.get('Data', [])
        if isinstance(datos_valores, list):
            for valor_obj in datos_valores:
                if isinstance(valor_obj, dict):
                    periodo = valor_obj.get('NombrePeriodo')
                    valor = valor_obj.get('Valor')
                    if periodo is not None and valor is not None:
                        registros_deciles.append({
                            'Indicador': indicador, 'Decil': decil,
                            'Año': int(periodo), 'Valor': float(valor)
                        })

df_renta_decil_raw = pd.DataFrame(registros_deciles)
df_renta_decil = df_renta_decil_raw.pivot_table(
    index=['Decil', 'Año'], columns='Indicador', values='Valor', aggfunc='first'
).reset_index()
df_renta_decil.columns.name = None

mapeo_columnas = {'Renta media': 'Media', 'Renta mediana': 'Mediana'}
df_renta_decil = df_renta_decil.rename(columns={col: mapeo_columnas.get(col, col) for col in df_renta_decil.columns})

print(f"  > Completado. Registros: {len(df_renta_decil)}")


  > Completado. Registros: 176


### 👥 9. Población por Edad, Sexo y Nacionalidad (INE - Tabla 56936)

**¿Qué es?**  
Datos del Padrón Municipal mostrando población residente desglosada por edad, sexo y nacionalidad.

**¿Para qué sirve?**  
- Calcular cifras absolutas: Convertir tasas AROPE (%) en número real de personas afectadas
- Contextualizar indicadores: "12.4 millones en AROPE" vs "26.5%"

**Transformación:** Datos trimestrales → Filtrar "Total" nacionalidad → Agrupar por año

In [8]:

tabla_poblacion = "56936"
url_tabla_poblacion = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{tabla_poblacion}"
response_poblacion = requests.get(url_tabla_poblacion)

if response_poblacion.status_code != 200:
    print(f"  ❌ Error al obtener datos: HTTP {response_poblacion.status_code}")
    df_poblacion = pd.DataFrame(columns=['Año', 'Sexo', 'Edad', 'Poblacion'])
else:
    try:
        data_poblacion_raw = response_poblacion.json()
        registros_poblacion = []

        mapeo_edad = {
            'De 0 a 4 años': '0-4', 'De 5 a 9 años': '5-9', 'De 10 a 14 años': '10-14',
            'De 15 a 19 años': '15-19', 'De 20 a 24 años': '20-24', 'De 25 a 29 años': '25-29',
            'De 30 a 34 años': '30-34', 'De 35 a 39 años': '35-39', 'De 40 a 44 años': '40-44',
            'De 45 a 49 años': '45-49', 'De 50 a 54 años': '50-54', 'De 55 a 59 años': '55-59',
            'De 60 a 64 años': '60-64', 'De 65 a 69 años': '65-69', 'De 70 a 74 años': '70-74',
            'De 75 a 79 años': '75-79', 'De 80 a 84 años': '80-84', 'De 85 a 89 años': '85-89',
            'De 90 a 94 años': '90-94', '95 y más años': '95+', '90 y más años': '90+'
        }

        for record in data_poblacion_raw:
            if not isinstance(record, dict): continue
            
            nombre = record.get('Nombre', '')
            partes = [p.strip() for p in nombre.split('.')]
            if len(partes) < 4: continue
            
            nacionalidad_raw = partes[1]
            edad_raw = partes[2]
            sexo_raw = partes[3]
            
            if nacionalidad_raw != 'Total': continue
            
            sexo = 'Hombres' if 'Hombres' in sexo_raw else 'Mujeres' if 'Mujeres' in sexo_raw else 'Total'
            edad = mapeo_edad.get(edad_raw, edad_raw)
            
            datos_valores = record.get('Data', [])
            if isinstance(datos_valores, list):
                for valor_obj in datos_valores:
                    if isinstance(valor_obj, dict):
                        periodo_raw = valor_obj.get('NombrePeriodo', valor_obj.get('Anyo'))
                        valor = valor_obj.get('Valor')
                        
                        if periodo_raw is not None and valor is not None:
                            try:
                                anyo = int(str(periodo_raw).split('T')[0]) if 'T' in str(periodo_raw) else int(periodo_raw)
                            except:
                                continue
                            
                            registros_poblacion.append({
                                'Año': anyo, 'Sexo': sexo, 'Edad': edad, 'Poblacion': float(valor)
                            })

        df_poblacion_raw = pd.DataFrame(registros_poblacion)
        if not df_poblacion_raw.empty:
            df_poblacion = df_poblacion_raw.groupby(['Año', 'Sexo', 'Edad'], as_index=False)['Poblacion'].mean()
        else:
            df_poblacion = pd.DataFrame(columns=['Año', 'Sexo', 'Edad', 'Poblacion'])

        print(f"  > Completado. Registros: {len(df_poblacion)}")
    except Exception as e:
        print(f"  ❌ Error procesando datos: {e}")
        df_poblacion = pd.DataFrame(columns=['Año', 'Sexo', 'Edad', 'Poblacion'])


  > Completado. Registros: 1380


### 📊 10. Población por CCAA, Edad y Sexo (INE - Tabla 66014)

**¿Qué es?**  
Población residente por comunidades autónomas, edad (grupos quinquenales) y sexo.

**¿Para qué sirve?**  
- Análisis demográfico regional
- Cálculo de cifras absolutas de AROPE por CCAA
- Heatmap CCAA × Edad

**Transformación:** Datos trimestrales → Agrupar por año → Población por CCAA/Edad/Sexo

In [9]:

codigo_pob_ccaa = "66014"
url_pob_ccaa = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{codigo_pob_ccaa}"
response_pob_ccaa = requests.get(url_pob_ccaa)

if response_pob_ccaa.status_code != 200:
    print(f"  ❌ Error al obtener datos: HTTP {response_pob_ccaa.status_code}")
    df_poblacion_ccaa_edad = pd.DataFrame(columns=['Año', 'CCAA', 'Sexo', 'Edad', 'Poblacion'])
else:
    try:
        data_pob_ccaa = response_pob_ccaa.json()
        registros_pob_ccaa = []
        
        ccaa_list = ['Total Nacional', 'Andalucía', 'Aragón', 'Asturias, Principado de', 
                     'Balears, Illes', 'Canarias', 'Cantabria', 'Castilla y León', 
                     'Castilla - La Mancha', 'Cataluña', 'Comunitat Valenciana', 
                     'Extremadura', 'Galicia', 'Madrid, Comunidad de', 'Murcia, Región de', 
                     'Navarra, Comunidad Foral de', 'País Vasco', 'Rioja, La', 'Ceuta', 'Melilla']
        
        sexo_list = ['Ambos sexos', 'Hombres', 'Mujeres']
        
        for serie in data_pob_ccaa:
            nombre = serie.get('Nombre', '')
            datos = serie.get('Data', [])
            partes = [p.strip() for p in nombre.split('.') if p.strip()]
            
            ccaa = None
            sexo = None
            edad = None
            
            for parte in partes:
                if any(c in parte for c in ccaa_list):
                    ccaa = parte
                elif any(s in parte for s in sexo_list):
                    sexo = parte
                elif 'Personas' not in parte and parte not in ['Total Nacional', 'Ambos sexos', 'Hombres', 'Mujeres']:
                    edad = parte
            
            if ccaa and sexo and edad:
                for punto in datos:
                    anyo_raw = punto.get('Anyo')
                    valor = punto.get('Valor')
                    
                    if anyo_raw and valor:
                        anyo = int(str(anyo_raw)[:4])
                        registros_pob_ccaa.append({
                            'Año': anyo, 'CCAA': ccaa, 'Sexo': sexo, 'Edad': edad, 'Poblacion': float(valor) * 1000
                        })

        df_poblacion_ccaa_edad = pd.DataFrame(registros_pob_ccaa)
        print(f"  > Completado. Registros: {len(df_poblacion_ccaa_edad)}")
    except Exception as e:
        print(f"  ❌ Error procesando: {e}")
        df_poblacion_ccaa_edad = pd.DataFrame(columns=['Año', 'CCAA', 'Sexo', 'Edad', 'Poblacion'])


  > Completado. Registros: 10260


### 📊 11. AROPE por CCAA (INE - Tabla 29288)

**¿Qué es?**  
Riesgo de pobreza o exclusión social por comunidades autónomas.

**¿Para qué sirve?**  
- Identificar "puntos negros" regionales
- Análisis de vulnerabilidad territorial

**Transformación:** Datos por CCAA/indicador → Filtrado AROPE → Tasas por CCAA

In [10]:

codigo_arope_ccaa = "29288"
url_arope_ccaa = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{codigo_arope_ccaa}"
response_arope_ccaa = requests.get(url_arope_ccaa)

if response_arope_ccaa.status_code != 200:
    print(f"  ❌ Error al obtener datos: HTTP {response_arope_ccaa.status_code}")
    df_arope_ccaa_filtrado = pd.DataFrame(columns=['Año', 'CCAA', 'Indicador', 'Valor'])
else:
    try:
        data_arope_ccaa = response_arope_ccaa.json()
        registros_arope_ccaa = []
        
        for serie in data_arope_ccaa:
            nombre = serie.get('Nombre', '')
            datos = serie.get('Data', [])
            partes = [p.strip() for p in nombre.split('.')]
            
            if len(partes) >= 3:
                ccaa = partes[0]
                indicador = partes[2]
                
                for punto in datos:
                    anyo_raw = punto.get('Anyo')
                    valor = punto.get('Valor')
                    
                    if anyo_raw and valor:
                        anyo = int(str(anyo_raw)[:4])
                        registros_arope_ccaa.append({
                            'Año': anyo, 'CCAA': ccaa, 'Indicador': indicador, 'Valor': float(valor)
                        })

        df_arope_ccaa = pd.DataFrame(registros_arope_ccaa)
        
        if not df_arope_ccaa.empty:
            df_arope_ccaa_filtrado = df_arope_ccaa[
                df_arope_ccaa['Indicador'].str.contains('AROPE|riesgo de pobreza|exclusión social', case=False, na=False)
            ]
            if df_arope_ccaa_filtrado.empty:
                df_arope_ccaa_filtrado = df_arope_ccaa.copy()
        else:
            df_arope_ccaa_filtrado = pd.DataFrame(columns=['Año', 'CCAA', 'Indicador', 'Valor'])

        print(f"  > Completado. Registros: {len(df_arope_ccaa_filtrado)}")
    except Exception as e:
        print(f"  ❌ Error procesando: {e}")
        df_arope_ccaa_filtrado = pd.DataFrame(columns=['Año', 'CCAA', 'Indicador', 'Valor'])


  > Completado. Registros: 680


### 📊 12. Gasto Medio por Hogar por Quintil - EPF (INE - Tabla 24900)

**¿Qué es?**  
Gasto medio por hogar y distribución del gasto por grupos de gasto y por quintiles.

**Fuente:** Encuesta de Presupuestos Familiares (EPF)

**¿Para qué sirve?**  
- Estructura de gasto por nivel de renta (quintiles)
- Base para análisis de inflación diferencial

**Transformación:** Datos por quintil/grupo_gasto → Porcentajes de gasto → EPF estructurada

In [11]:

codigo_epf = "24900"
url_epf = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{codigo_epf}"
response_epf = requests.get(url_epf)

if response_epf.status_code != 200:
    print(f"  ❌ Error al obtener datos: HTTP {response_epf.status_code}")
    df_epf_gasto = pd.DataFrame(columns=['Año', 'Quintil', 'Grupo_Gasto', 'Tipo_Valor', 'Valor'])
else:
    try:
        data_epf = response_epf.json()
        registros_epf = []
        
        for serie in data_epf:
            nombre = serie.get('Nombre', '')
            datos = serie.get('Data', [])
            
            quintil = 'Total'
            quintil_match = re.search(r'Quintil\s*(\d+)', nombre, re.IGNORECASE)
            if quintil_match:
                quintil = f'Q{quintil_match.group(1)}'
            
            tipo_valor = 'Gasto_Medio'
            if 'Dato base.' in nombre:
                partes_despues = nombre.split('Dato base.')[-1]
                partes_sin_quintil = re.sub(r'Quintil\s*(\d+|Total)\s*\.?\s*$', '', partes_despues).strip()
                
                if 'Gasto medio por hogar' in partes_sin_quintil:
                    tipo_valor = 'Gasto_Hogar'
                elif 'Gasto medio por persona' in partes_sin_quintil:
                    tipo_valor = 'Gasto_Persona'
                elif 'Distribución (porcentajes horizontales)' in partes_sin_quintil:
                    tipo_valor = 'Distribucion_H'
                elif 'Distribución (porcentajes verticales)' in partes_sin_quintil:
                    tipo_valor = 'Distribucion_V'
            
            grupo_gasto = 'Índice_General'
            if 'Total Nacional.' in nombre and 'Dato base.' in nombre:
                entre_nacional_y_dato = nombre.split('Total Nacional.')[-1].split('Dato base.')[0].strip()
                grupo_gasto = entre_nacional_y_dato.strip().replace(' ', '_')
            
            for punto in datos:
                anyo_raw = punto.get('Anyo')
                valor = punto.get('Valor')
                
                if anyo_raw and valor is not None:
                    anyo = int(str(anyo_raw)[:4])
                    registros_epf.append({
                        'Año': anyo, 'Quintil': quintil, 'Grupo_Gasto': grupo_gasto,
                        'Tipo_Valor': tipo_valor, 'Valor': float(valor)
                    })

        df_epf_gasto = pd.DataFrame(registros_epf)
        print(f"  > Completado. Registros: {len(df_epf_gasto)}")
    except Exception as e:
        print(f"  ❌ Error procesando: {e}")
        df_epf_gasto = pd.DataFrame(columns=['Año', 'Quintil', 'Grupo_Gasto', 'Tipo_Valor', 'Valor'])


  > Completado. Registros: 5616


### 📊 13. IPC Sectorial por Grupos ECOICOP (INE - Tabla 50902)

**¿Qué es?**  
Índices de precios de consumo por grupos ECOICOP (clasificación de consumo).

**¿Para qué sirve?**  
- Calcular inflación diferencial por quintil de renta
- Cruzar con estructura de gasto de EPF

**Transformación:** IPC por grupo ECOICOP → Variación anual por categoría

In [12]:

codigo_ipc_sect = "50902"
url_ipc_sect = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{codigo_ipc_sect}"
response_ipc_sect = requests.get(url_ipc_sect)

if response_ipc_sect.status_code != 200:
    print(f"  ❌ Error al obtener datos: HTTP {response_ipc_sect.status_code}")
    df_ipc_sectorial_anual = pd.DataFrame(columns=['Año', 'Categoria_ECOICOP', 'IPC_Indice', 'Inflacion_Sectorial_%'])
else:
    try:
        data_ipc_sect = response_ipc_sect.json()
        registros_ipc_sect = []
        
        for serie in data_ipc_sect:
            nombre = serie.get('Nombre', '')
            datos = serie.get('Data', [])
            nombre_limpio = nombre.strip()
            
            if nombre_limpio.startswith('Total Nacional. '):
                resto = nombre_limpio.replace('Total Nacional. ', '')
                partes = resto.rsplit('. ', 1)
                
                if len(partes) == 2:
                    categoria = partes[0].strip()
                    tipo_metrica = partes[1].rstrip('. ')
                    
                    for dato in datos:
                        nombre_periodo = dato.get('NombrePeriodo')
                        
                        if nombre_periodo is not None:
                            registros_ipc_sect.append({
                                'Año': int(nombre_periodo), 
                                'Categoria_ECOICOP': categoria,
                                'Tipo_Metrica': tipo_metrica,
                                'IPC_Indice': float(dato['Valor'])
                            })
        
        df_ipc_sectorial = pd.DataFrame(registros_ipc_sect)

        if not df_ipc_sectorial.empty:
            df_ipc_sectorial_anual = df_ipc_sectorial.groupby(
                ['Año', 'Categoria_ECOICOP', 'Tipo_Metrica'], as_index=False
            )['IPC_Indice'].mean()

            df_ipc_sectorial_anual = df_ipc_sectorial_anual.sort_values(
                by=['Categoria_ECOICOP', 'Tipo_Metrica', 'Año']
            )
            
            df_ipc_sectorial_anual['Inflacion_Sectorial_%'] = df_ipc_sectorial_anual.apply(
                lambda row: row['IPC_Indice'] if row['Tipo_Metrica'] == 'Variación anual' else pd.NA, axis=1
            )

            print(f"  > Completado. Registros: {len(df_ipc_sectorial_anual)}")
        else:
            df_ipc_sectorial_anual = pd.DataFrame(columns=['Año', 'Categoria_ECOICOP', 'IPC_Indice', 'Inflacion_Sectorial_%'])

    except Exception as e:
        print(f"  ❌ Error procesando: {e}")
        df_ipc_sectorial_anual = pd.DataFrame(columns=['Año', 'Categoria_ECOICOP', 'IPC_Indice', 'Inflacion_Sectorial_%'])


### 💾 14. Guardar DataFrames en Pickle

Guardado de los 14 DataFrames del INE en archivos pickle para carga posterior en 01c.

In [13]:
print("💾 Guardando DataFrames INE en pickle...")

dataframes_ine = {
    'df_ipc_anual': df_ipc_anual,
    'df_umbral_limpio': df_umbral_limpio,
    'df_carencia_material': df_carencia_material,
    'df_arope_edad_sexo': df_arope_edad_sexo,
    'df_arope_hogar': df_arope_hogar,
    'df_arope_laboral': df_arope_laboral,
    'df_gini_ccaa': df_gini_ccaa,
    'df_renta_decil': df_renta_decil,
    'df_poblacion': df_poblacion,
    'df_poblacion_ccaa_edad': df_poblacion_ccaa_edad,
    'df_arope_ccaa': df_arope_ccaa_filtrado,
    'df_epf_gasto': df_epf_gasto,
    'df_ipc_sectorial': df_ipc_sectorial_anual
}

for nombre, df in dataframes_ine.items():
    ruta = CACHE_DIR / f'{nombre}.pkl'
    with open(ruta, 'wb') as f:
        pickle.dump(df, f)
    print(f'  ✅ {nombre}: {len(df)} registros → {ruta.name}')

print(f'\n✅ Total guardados: {len(dataframes_ine)} pickles INE')
print(f'🕒 Fin: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')


💾 Guardando DataFrames INE en pickle...
  ✅ df_ipc_anual: 24 registros → df_ipc_anual.pkl
  ✅ df_umbral_limpio: 32 registros → df_umbral_limpio.pkl
  ✅ df_carencia_material: 1683 registros → df_carencia_material.pkl
  ✅ df_arope_edad_sexo: 408 registros → df_arope_edad_sexo.pkl
  ✅ df_arope_hogar: 308 registros → df_arope_hogar.pkl
  ✅ df_arope_laboral: 152 registros → df_arope_laboral.pkl
  ✅ df_gini_ccaa: 340 registros → df_gini_ccaa.pkl
  ✅ df_renta_decil: 176 registros → df_renta_decil.pkl
  ✅ df_poblacion: 1380 registros → df_poblacion.pkl
  ✅ df_poblacion_ccaa_edad: 10260 registros → df_poblacion_ccaa_edad.pkl
  ✅ df_arope_ccaa: 680 registros → df_arope_ccaa.pkl
  ✅ df_epf_gasto: 5616 registros → df_epf_gasto.pkl
  ✅ df_ipc_sectorial: 0 registros → df_ipc_sectorial.pkl

✅ Total guardados: 13 pickles INE
🕒 Fin: 2025-11-16 14:14:43
